First, import needed modules

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
from geopy.geocoders import Nominatim
import os

Initalize variables

In [ ]:
path = os.getcwd()
path

Load Dataset which has the endpoints of power line projects

In [ ]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [ ]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations

In [5]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations

KeyboardInterrupt: 

Merge into powerlines dataset

In [ ]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_address', 'Start_geometry', 'End', 'End_address', 'End_geometry', 'direct_line', 'New', 'Retro'])
powerlines_geo

Create direct line from start and end coordinates

In [ ]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = LineString([start, end])
powerlines_geo.head()

Export shapefile

In [ ]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG2013', 'Start', 'End', 'geometry', 'New', 'Retro'])
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')